In [93]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from time import time

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

v = TfidfVectorizer()
X_train = v.fit_transform(train['comment_text'])
X_test = v.transform(test['comment_text'])



In [8]:
for label in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    y = train[label]
    model = LogisticRegression(n_jobs=-1)
    model.fit(X_train, y)
    test[label] = model.predict_proba(X_test)[:, 1]
    
test.drop('comment_text', axis=1, inplace=True)
test.to_csv('simplest.csv', index=False)

/home/shj16110/anaconda2/envs/new2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_auc_score

In [13]:
df_train, df_test = train_test_split(train, test_size=0.2)
df_train.shape, df_test.shape

((127656, 8), (31915, 8))

In [15]:
ID_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [17]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [21]:
for i in ID_cols:
    print(train[i].value_counts())

0    144277
1     15294
Name: toxic, dtype: int64
0    157976
1      1595
Name: severe_toxic, dtype: int64
0    151122
1      8449
Name: obscene, dtype: int64
0    159093
1       478
Name: threat, dtype: int64
0    151694
1      7877
Name: insult, dtype: int64
0    158166
1      1405
Name: identity_hate, dtype: int64


In [24]:
pd.DataFrame(columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

,toxic,severe_toxic,obscene,threat,insult,identity_hate


In [ ]:
%%time
pred_test = pd.DataFrame(columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

v = TfidfVectorizer()
Xf_train = v.fit_transform(df_train['comment_text'])
Xf_test = v.transform(df_test['comment_text'])

score=[]
for label in ID_cols:
    start=time()
    y = df_train[label]


    
    model = RandomForestClassifier(n_estimators = 300, min_samples_split=3 , n_jobs=-1)
    model.fit(Xf_train, y)
    pred_test[label] = model.predict_proba(Xf_test)[:, 1]
    
    s = roc_auc_score(df_test[label],pred_test[label])
    score.append(s)

    print (label, 'roc_auc: ',s, 'used {:.2f} s'.format(time()-start))

print(sum(score)/len(score))

In [88]:
Xf_train = v.transform(df_train['comment_text'])
Xf_test = v.transform(df_test['comment_text'])

def helper(func = LogisticRegression()):
    
    pred_test = pd.DataFrame(columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

    score=[]
    for label in ID_cols:
        start=time()

        y = df_train[label]

        model = func
        model.fit(Xf_train, y)
        pred_test[label] = model.predict_proba(Xf_test)[:, 1]

        s = roc_auc_score(df_test[label],pred_test[label])
        score.append(s)

        print (label, 'roc_auc: ',s, 'used {:.2f} s'.format(time()-start))

    print(sum(score)/len(score))
    return model

In [89]:
model = helper(LogisticRegression(C=4, n_jobs=-1))

('toxic', 'roc_auc: ', 0.9745753352859406, 'used 3.28 s')
('severe_toxic', 'roc_auc: ', 0.987373458384737, 'used 2.94 s')
('obscene', 'roc_auc: ', 0.9857250678030458, 'used 2.52 s')
('threat', 'roc_auc: ', 0.9885834251895284, 'used 2.14 s')
('insult', 'roc_auc: ', 0.9798939463930224, 'used 3.06 s')
('identity_hate', 'roc_auc: ', 0.9750321932631231, 'used 2.42 s')
0.9818639043865662


In [94]:
for i in ID_cols:
    test[i] = model.predict_proba(X_test)[:, 1]
    
test.drop('comment_text', axis=1, inplace=True)
test.to_csv('simplest2.csv', index=False)